In [ ]:
# L 4-23-25
# notebooks/3.1_Generate_Spectrograms.ipynb

In [ ]:
# notebooks/3.1_Generate_Spectrograms.ipynb

import gc
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib

matplotlib.use('Agg')  # Non-interactive backend

# Default parameters for standalone run
USED_TRACKS_CSV = "../reports/2_MFCC_RF_Classifier/used_tracks.csv"
AUDIO_DIR = "../data/fma_small"
DEFAULT_OUTPUT_DIR = "../spectrograms"
SONG_INFERENCE_DIR = "../data/Songs/"
SONG_OUTPUT_DIR = "../reports/4_Classify_New_Song/spectrograms/"
IMG_SIZE = 128


def generate_spectrogram(input_path, output_path, img_size=128):
    try:
        y, sr = librosa.load(input_path, sr=None, duration=30)
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_dB = librosa.power_to_db(S, ref=np.max)

        fig, ax = plt.subplots(figsize=(1.28, 1.28), dpi=100)
        librosa.display.specshow(S_dB, sr=sr, cmap='viridis', ax=ax)
        ax.axis('off')
        plt.tight_layout(pad=0)
        fig.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        with Image.open(output_path) as img:
            img = img.convert("RGB")
            img = img.resize((img_size, img_size))
            img.save(output_path)

        del y, sr, S, S_dB
        gc.collect()
        return True
    except Exception as e:
        print(f"Failed to generate spectrogram for {input_path}: {e}")
        gc.collect()
        return False


def generate_spectrograms_from_csv(csv_path, audio_dir, output_dir, img_size=128):
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} tracks from {csv_path}")
    for _, row in df.iterrows():
        track_id = str(row['track_id']).zfill(6)
        genre = row['genre']
        subfolder = track_id[:3]
        input_path = os.path.join(audio_dir, subfolder, f"{track_id}.mp3")
        genre_folder = os.path.join(output_dir, genre)
        os.makedirs(genre_folder, exist_ok=True)
        output_path = os.path.join(genre_folder, f"{track_id}.png")
        if os.path.exists(output_path):
            continue
        generate_spectrogram(input_path, output_path, img_size)


def generate_spectrograms_from_folder(song_folder, output_dir, img_size=128):
    for genre in os.listdir(song_folder):
        genre_dir = os.path.join(song_folder, genre)
        if not os.path.isdir(genre_dir):
            continue

        output_genre_dir = os.path.join(output_dir, genre)
        os.makedirs(output_genre_dir, exist_ok=True)

        for fname in os.listdir(genre_dir):
            if not fname.endswith(".mp3"):
                continue
            fpath = os.path.join(genre_dir, fname)
            output_path = os.path.join(
                output_genre_dir, f"{os.path.splitext(fname)[0]}.png")
            if os.path.exists(output_path):
                continue
            generate_spectrogram(fpath, output_path, img_size)


# Only execute spectrogram generation when running this notebook directly
if __name__ == "__main__" or "__file__" not in globals():
    # Called by another notebook (like classify_new_song)
    if os.path.exists(SONG_INFERENCE_DIR):
        generate_spectrograms_from_folder(
            SONG_INFERENCE_DIR, SONG_OUTPUT_DIR, IMG_SIZE)
    else:
        generate_spectrograms_from_csv(
            USED_TRACKS_CSV, AUDIO_DIR, DEFAULT_OUTPUT_DIR, IMG_SIZE)

Loaded 7994 usable tracks from CSV
